In [1]:
# import pandas
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [56]:
book_ratings = pd.read_csv('/content/drive/MyDrive/data exercises/W10/BX-Book-Ratings.csv',sep=";", encoding="latin").sample(frac=.05)
book_ratings = book_ratings[book_ratings['Book-Rating'] > 0] # Remove books with rating of zero since we don't want to recommend these
book_ratings

,User-ID,ISBN,Book-Rating
422341,100906,0345378482,8
1022060,245001,2264035870,4
107185,24641,089659419X,8
1131441,271306,0451458583,7
370469,88733,1569874115,10
...,...,...,...
1105328,265213,0060959606,7
12155,1025,0805067868,10
260050,60244,0375702709,9
80212,16795,0684871262,9


In [57]:
book_ratings['Book-Rating'].value_counts()

8     5081
10    3947
7     3831
9     3337
5     2557
6     1870
4      426
3      316
2      120
1       96
Name: Book-Rating, dtype: int64

In [12]:
books = pd.read_csv('/content/drive/MyDrive/data exercises/W10/BX-Books.csv',sep=";", encoding="latin", on_bad_lines='skip', engine='python')
print(books.shape)
books.head()

(270491, 8)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [58]:
df_book_features = book_ratings.merge(
    books[['ISBN', 'Book-Title']], left_on='ISBN', right_on='ISBN',
).set_index('ISBN')
print(df_book_features.shape)
df_book_features

(19026, 3)


,User-ID,Book-Rating,Book-Title
ISBN,,,
0345378482,100906,8,The Andromeda Strain
0345378482,142904,7,The Andromeda Strain
0345378482,252695,9,The Andromeda Strain
0345378482,224138,10,The Andromeda Strain
0345378482,171295,9,The Andromeda Strain
...,...,...,...
0804111499,216444,5,Single Solutions: An Essential Guide for the S...
0449243141,8680,5,Restoring the American Dream
0060959606,265213,7,Use Me : Fiction


In [59]:
# Put the data in a format that can be used for NearestNeighbors
book_rating = df_book_features.pivot_table(index='ISBN', columns='User-ID', values='Book-Rating')
print(book_rating.shape)
book_rating.head()

(14852, 10092)


User-ID,44,92,99,202,242,243,254,300,302,387,...,278554,278582,278586,278633,278668,278818,278843,278851,278852,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
0001053744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0002226162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0002240114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0002243962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0002251760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# ensure there are no duplicate ISBNs
book_rating.index.value_counts(ascending=False)

0001053744    1
0743403452    1
0743250532    1
0743252101    1
0743255259    1
             ..
0425174271    1
0425174344    1
0425174433    1
0425174808    1
9990591199    1
Name: ISBN, Length: 14852, dtype: int64

In [60]:
book_rating = book_rating.fillna(0)
book_rating.head()

User-ID,44,92,99,202,242,243,254,300,302,387,...,278554,278582,278586,278633,278668,278818,278843,278851,278852,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
0001053744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002226162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002240114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002243962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002251760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
from scipy.sparse import csr_matrix
book_rating2 = csr_matrix(book_rating)

In [63]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=5)

neigh.fit(book_rating)

NearestNeighbors()

In [62]:
neigh2 = NearestNeighbors(n_neighbors=5)

neigh2.fit(book_rating2)

NearestNeighbors()

# With dense matrix

In [66]:
def recommend(book_title, df=df_book_features, rating_matrix = book_rating):
  isbn = df[df['Book-Title'] == book_title].index
  
  distance, neighbors = neigh2.kneighbors(rating_matrix.loc[isbn])
  recommendations = []
  for neighbor in neighbors[0]:
    isbn = rating_matrix.iloc[neighbor].name
    recommendations.append(df.loc[isbn, 'Book-Title'])
  return recommendations


In [67]:
recommend('The Andromeda Strain')

[ISBN
 0345378482    The Andromeda Strain
 0345378482    The Andromeda Strain
 0345378482    The Andromeda Strain
 0345378482    The Andromeda Strain
 0345378482    The Andromeda Strain
 Name: Book-Title, dtype: object,
 'Amsterdam : A Novel',
 'Rush Limbaugh Is a Big Fat Idiot: And Other Observations',
 'How to Live Cat',
 "Jessie's Father (Harlequin Superromance, 851)"]

In [68]:
recommend('Restoring the American Dream')

['Restoring the American Dream',
 'The Silicon Mage (Windrose Chronicles, Book 2)',
 'The Case of Cabin 13: A John Darnell Mystery (John Darnell Mysteries)',
 'Thirteen Days: A Memoir of the Cuban Missile Crisis',
 'The Magic Circle']

In [70]:
recommend('Use Me : Fiction')

['Use Me : Fiction',
 'Damned in Paradise: A Nathan Heller Novel (Nathan Heller Novels (Paperback))',
 'Who Was the Man in the Iron Mask?: And Other Historical Enigmas (Penguin Classic History S.)',
 'Talking Rain: A Professo (Professor Teodora Morelli Mystery)',
 'I Know You, Joanna (Black Lace)']

In [72]:
recommend("Humphrey's Corner")

['The Grapes of Wrath: John Steinbeck Centennial Edition (1902-2002)',
 "Humphrey's Corner",
 "Prophecies: Visions of the World's Fate : Truths, Possibilities, or Fallacies?",
 'The Society Bride (Harlequin Presents, 2388)',
 'The Hand of Ethelberta: A Comedy in Chapters (Penguin Classics)']